# Jugant amb GPT i funcions
Marc Alier Juny 2023
https://wasabi.essi.upc.edu/ludo 


## Configuració


In [ ]:
#potser es necessari
%pip install --upgrade openai

### Importa la llibreria openai i carrega la teva API Key

In [2]:
# importa la llibreria OpenAI
import openai
import json
import requests


from IPython.display import display, HTML

with open('..//mykey.json', 'r') as f:
    data = json.load(f)
    print("OK")
    
openai.api_key = data['key']

# alternativament pots fer 
#
# openai.api_key = "posa aqui la teva clau"


OK


## Definim el model al que ens conectem

In [3]:
#MODEL = "gpt-3.5-turbo"
MODEL = "gpt-4-0613" # model minim per tenir funcions

## Què és una funció en el context de GPT?

En el context de l'API de GPT (GPT-3.5 turbo i GPT-4 a partir de 17 juny 2023) una funció és una **possibilitat que oferim al model per a respondre a les indicacions d'un usuari.** 

El desenvolupador descriu en llenguatge natural el que suposadament fa una funció i els paràmetres que rep aquesta funció en JSON. 

Si el model decideix que per a donar una bona resposta necessita invocar aquesta funció, en comptes de donar una resposta estàndard ens **retornarà una sol·licitud d'execució d'aquesta funció juntament amb els valors dels paràmetres.** 

### Funció per cridar a GPT
(No podem fer servir la funció del package "openai", perque no suporta el paràmetre "functions" encara)



In [4]:
def crida_a_openai(model, messages, temperature):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {openai.api_key}'
    }

    data = {
        'model': model,
        'messages': messages,
        'temperature': temperature
    }

    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, data=json.dumps(data))

    return response.json()

## Exemple de crida a GPT 

In [5]:
response = crida_a_openai(
    model=MODEL,
    messages=[
        {"role": "system", "content": "Ets un dels tres porquets del \
        conte i ets a casa."},
        {"role": "user", "content": "Pom Pom."},
        {"role": "assistant", "content": "Quí hi ha?"},
        {"role": "user", "content": "Soc el llop ferotge,\
         obre la porta o bufaré, bufaré i la casa ensorraré."},
    ],
    temperature=0,
)

response

{'id': 'chatcmpl-7UyQtHBb4rZCaPVmNBq67g6ijtZLD',
 'object': 'chat.completion',
 'created': 1687616499,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'No, no, no! No obriré la porta! La meva casa està feta de maons, no la podràs ensorrar bufant.'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 74, 'completion_tokens': 35, 'total_tokens': 109}}

In [6]:
# Obtenir el contingut del message a la primera choice
html_content = response['choices'][0]['message']['content']

# mostrar contingut missatge
print("GTP:")
display(HTML(html_content))

## Ara amb funcions

Anem a proposar dues funcions disponibles pel porquet.

In [15]:
def crida_a_openai_amb_funcions(model, messages, functions, temperature):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {openai.api_key}'
    }

    data = {
        'model': model,
        'messages': messages,
        'functions': functions,
        'temperature': temperature
    }

    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, data=json.dumps(data))

    return response.json()


In [30]:
functions = [
    {
        "name": "avisa_al_cacador",
        "description": "Avisa al caçador per demanar ajuda",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "La ubicació de la casa del porquet."
                },
                "porquet": {
                    "type": "string",
                    "description": "El nom del porquet que demana ajuda."
                },
                "urgent": {
                    "type": "boolean",
                    "description": "Indica si l'ajuda és urgent o no."
                }
            },
            "required": ["location", "porquet", "urgent"]
        },
        "results": {
            "type": "string",
            "description": "Un missatge que descriu la situació."
        }
    },
    {
        "name": "comunica_amb_germa",
        "description": "Permet al porquet comunicar-se amb un dels seus germans",
        "parameters": {
            "type": "object",
            "properties": {
                "porquet": {
                    "type": "string",
                    "description": "El nom del porquet que vol comunicar-se."
                },
                "germa": {
                    "type": "string",
                    "description": "El nom del germa amb qui el porquet vol comunicar-se."
                },
                "missatge": {
                    "type": "string",
                    "description": "El missatge que el porquet vol enviar al seu germa."
                }
            },
            "required": ["porquet", "germa", "missatge"]
        },
        "results": {
            "type": "string",
            "description": "Un missatge confirmant que el missatge ha estat enviat al germa."
        }
    }
]


In [36]:
messages=[
        {"role": "system", "content": "Ets un dels tres porquets del \
        conte i ets a casa."},
        {"role": "user", "content": "Pom Pom."},
        {"role": "assistant", "content": "Quí hi ha?"},
        {"role": "user", "content": "Soc el llop ferotge,\
         obre la porta o bufaré, bufaré i la casa ensorraré."},
    ]

response = crida_a_openai_amb_funcions(
    model=MODEL,
    messages=messages,
    functions=functions,
    temperature=0,
)

response

{'id': 'chatcmpl-7Uz2CVGH8FnmyBiXdwJ0lMOSsZohX',
 'object': 'chat.completion',
 'created': 1687618812,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'avisa_al_cacador',
     'arguments': '{\n  "location": "casa del porquet",\n  "porquet": "porquet",\n  "urgent": true\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 258, 'completion_tokens': 38, 'total_tokens': 296}}

### Tractant el retorn de GPT 

In [37]:
if ('choices' in response) and isinstance(response['choices'], list) and len(response['choices']) > 0:
        firstChoice = response['choices'][0]

        #Obtenim el parametre'finish_reason' i el posem a una variable

        motiu_de_parada = firstChoice.get('finish_reason', None)
        
        if motiu_de_parada == 'function_call':         
            nom_funcio = response['choices'][0]['message']['function_call']['name']
            print(f'GPT demana fer una Function call a {nom_funcio}')
            
        else : 
            # Obtenir el contingut del message a la primera choice
            html_content = response['choices'][0]['message']['content']

            # mostrar contingut missatge
            print("GTP:")
            display(HTML(html_content))
else : print("Error:no choices")


GPT demana fer una Function call a avisa_al_cacador


In [38]:
response

{'id': 'chatcmpl-7Uz2CVGH8FnmyBiXdwJ0lMOSsZohX',
 'object': 'chat.completion',
 'created': 1687618812,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'avisa_al_cacador',
     'arguments': '{\n  "location": "casa del porquet",\n  "porquet": "porquet",\n  "urgent": true\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 258, 'completion_tokens': 38, 'total_tokens': 296}}

In [43]:
def f_avisa_al_cacador(location, porquet):
    return f"Tranquil {porquet} vinc ben de pressa cap a {location} per ajudarte. \
    Digues-li al llop que si no marxa ara mateix, quan jo arribi li donaré una bona surra."
    
if nom_funcio == "avisa_al_cacador" :
    
    # response['choices'][0]['message']['function_call']['arguments'] és JSON no un diccionari de python
    
    arguments = json.loads(response['choices'][0]['message']['function_call']['arguments'])
    
    location = arguments['location']
    nom_porquet = arguments['porquet']

    resposta_funcio= f_avisa_al_cacador(location, nom_porquet)

    response_a_segona_crida = crida_a_openai(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Ets un dels tres porquets del \
            conte i ets a casa."},
            {"role": "user", "content": "Pom Pom."},
            {"role": "assistant", "content": "Quí hi ha?"},
            {"role": "user", "content": "Soc el llop ferotge,\
             obre la porta o bufaré, bufaré i la casa ensorraré."},
            {"role": "function", 
             "name":"avisa_al_cacador", 
             "content":resposta_funcio},
        ],
        temperature=1,
    )

else :
    print("cap funcio")
            

In [44]:
# Obtenir el contingut del message a la primera choice
html_content = response_a_segona_crida['choices'][0]['message']['content']

# mostrar contingut missatge
print("GTP:")
display(HTML(html_content))


GTP:
